#### 고려대한국어대사전 오픈프로 포멧으로 변경 (idiom)

1. 오픈프로 필요 기타 칼럼 추가
2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
4. 설명 부분의 < > 부분 용법 내용으로 분류 
5. 품사 폰트 변경
6. "¶" 이걸 기준으로 설명과 예시 분류
7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다.
8. 용례 여러개일 경우 분류
9. <> [] 제거 만들기  
10. 뜻풀이 참조어 링크 태그로 변환 + <유> 부분 태그 변환 (EID 정보 있을 경우)
11. 중복제거
12. 네이버와 아이디 매칭

In [110]:
import pandas as pd
import re
from time import time

start_time = time()

pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
symbols = ['①', '②', '③']

# input_file_path 부분을 채워주세요.
input_file_path = "../asset/idiom.csv"
target = pd.read_csv(input_file_path)

In [111]:
# 1. 오픈프로 필요한 기타 칼럼 추가 
target['type'] = '숙어'
target['typediff'] = '괄호 없앤 속담/관용구'
target['shoulder'] = int(0)
target['onum'] = int(1)
target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'shoulder', 
                'type', 'typediff', 'explan', 'onum']]

In [112]:
# 2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
LEN = len(target)
for i in range(LEN):
    target.loc[i, 'idiom'] = target.loc[i, 'idiom'][1:-1]

In [113]:
# 3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[0] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[1])
        target.loc[i, 'explan'] = temps[0][1:].strip()
        new_row = target.loc[i, :].copy()
        new_row['onum'] = 2
        new_row['explan'] = temps[1].strip()
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[2] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[2])
        target.loc[i, 'explan'] = temps[0].strip()
        new_row = target.loc[i, :].copy()
        new_row['explan'] = temps[1].strip()
        new_row['onum'] = 3
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

In [114]:
# 4. 설명 부분의 < > 부분 용법 내용으로 분류 
target['pattern'] = ''

LEN = len(target)
for i in range(LEN):
    p = r'\s*◀(.*?)▶' 
    m = re.search(p, target.loc[i, 'explan'])
    if m:
        result = m.group().strip() 
        target.loc[i, 'pattern'] = result[1:-1] 
        target.loc[i, 'explan'] = re.sub(p, '', target.loc[i, 'explan']).strip() 

In [115]:
# 5. 품사 폰트 변경
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'pattern'] == '':
        continue
    for pattern in pattern_format:
        target.loc[i, 'pattern'] = re.sub(pattern, pattern_format[pattern], target.loc[i, 'pattern'])

target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'shoulder', 'type', 'typediff', 'onum', 'pattern', 'explan']]

In [116]:
# 6. "¶" 이걸 기준으로 설명과 예시 분류
LEN = len(target)
target['explan'] = target['explan'].str.strip()
for i in range(LEN):
    temp = target.loc[i, 'explan'].split('¶')
    target.loc[i, 'explan'] = temp[0].strip()
    if len(temp) != 1:
        target.loc[i, 'example'] = temp[1].strip()

In [117]:
# 7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다. (뜻번호당 하나만!!! 용례당 하나가 아님)
target['example'].fillna('', inplace=True)
target['similar_expression'] = ''

LEN = len(target)
for i in range(LEN):
    if '<유>' in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<유>')
        target.loc[i, 'example'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip() 

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_22223/1795551707.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['example'].fillna('', inplace=True)


In [118]:
# 8. 용례 여러개일 경우 분류
LEN = len(target)
extra_list = []
for i in range(LEN):
    if '/' in target.loc[i, 'example']:
        temps = target.loc[i, 'example'].split(' / ')
        target.loc[i, 'example'] = temps[0].strip()
        for example in temps[1:]:
            new_row = target.loc[i, :].copy()
            new_row['example'] = temps[1].strip()
            extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'onum':[extra_list[i]['onum']], 'pattern':[extra_list[i]['pattern']], 
                            'explan':[extra_list[i]['explan']], 'example':extra_list[i]['example']})
    target = pd.concat([target, new_row], ignore_index=True)

target['similar_expression'].fillna('', inplace=True)
target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_22223/1837271683.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['similar_expression'].fillna('', inplace=True)


In [119]:
# 9. 뜻풀이 참조어 링크 태그로 변환 + <유>

In [120]:
# 10. 중복제거
condition = target.duplicated(subset=['uid', 'onum'], keep='first')
condition2 = target.duplicated(subset=['uid'], keep='first')
target.loc[condition, 'uid':'explan'] = ""
target.loc[condition2, 'uid':'typediff'] = ""

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_22223/819081721.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_22223/819081721.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition, 'uid':'explan'] = ""
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_22223/819081721.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[

In [121]:
# 11. 네이버와 아이디 매칭
target['naver_id'] = ""
target = target[['uid', 'naver_id', 'eid', 'ohw', 'ocol', 'idiom', 'shoulder', 'type', 'typediff',
                'onum', 'pattern', 'explan', 'example', 'similar_expression']]

ID = pd.read_csv('../asset/idiom_id.csv')
LEN = len(target)
LEN_ID = len(ID)
for i in range(LEN):
        target_idiom = target.loc[i, 'idiom']
        start = 0
        end = LEN_ID - 1
        while True:
                if start > end:
                        break
                mid = (start + end) // 2
                try_idiom = ID.loc[mid, 'idiom']
                if target_idiom == try_idiom:
                        target.loc[i, 'naver_id'] = ID.loc[mid, 'naver_id']
                        break
                elif target_idiom < try_idiom:
                        end = mid - 1
                else:
                        start = mid + 1

In [122]:
condition = (target['naver_id'] == "") & (target['uid'] != "")
# 신규어 개수 (매칭 안된 idiom)
print(f"신규(매칭 안됨) : {len(target[condition])}")

신규(매칭 안됨) : 43


In [123]:
target

,uid,naver_id,eid,ohw,ocol,idiom,shoulder,type,typediff,onum,pattern,explan,example,similar_expression
0,1,4a81f70b9f6046fe908cc538c9940a68,443,가꾸로,가꾸로박히다,가꾸로 박히다,0,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)에,(사람이 바닥에) 머리가 땅을 향한 채로 떨어지다.,아이가 침대에서 장난을 치다가 방바닥에 가꾸로 박혔다.,
1,2,032b6d148bd6465caf7ebf5789028e31,459,가난,가난이들다,가난이 들다,0,숙어,괄호 없앤 속담/관용구,1,(명사)에,(집이나 나라에) 어려움이 닥쳐 살림살이가 넉넉하지 못하게 되다.,부잣집에 가난이 드니 전과 달리 인심이 박해졌다.,
2,,,,,,,,,,2,(명사)이,(무엇이) 드물어 구하기 어렵게 되다.,요즘은 인재가 가난이 들었어요.,
3,,,,,,,,,,3,(명사)이,(농작물이) 여느 해보다 아주 적게 수확되다.,비가 제때 안 와서 배추가 가난이 들었다.,
4,4,cfb56075b9984f8e951f7131a2a1229e,583,가늠,가늠을보다,가늠(을) 보다〔잡다〕,0,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 목표나 형편을) 겨누어 보다.,강 이사가 이번 일의 성패를 가늠을 봤지만 결과를 정확히 예측할 수는 없었다.,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5602,5278,,185641,손꼽다,손꼽아기다리다,손꼽아 기다리다,0,숙어,괄호 없앤 속담/관용구,1,(명사)이 (명사)을,(사람이 어떤 날이나 일을) 초조한 마음으로 기다리다.,철수는 제대하는 날만을 손꼽아 기다렸다.,
5603,,,,,,,,,,,,,나는 어머니와 같이 외출하는 날만을 손꼽아 기다렸다.,
5604,5279,,69525,눈앞,눈앞이노래지다,눈앞이 노래지다〔누레지다〕,0,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 기력이 쇠하여 정신이 아찔하게 되다.,나는 수술 후유증이 남아서 조금만 무리를 해도 눈앞이 노래진다.,
5605,,,,,,,,,,2,(명사)이,(사람이) 지나치게 상심하여 어찌할 바를 모르다.,그때 우리는 아버지의 사고 소식을 듣고 눈앞이 노래지는 것 같았다.,


In [124]:
target.to_csv('../result/processed_idiom.csv', index=False)
end_time = time()
print(f"!!!FINISHED!!! 걸린시간 : {round(end_time - start_time, 2)}s")

!!!FINISHED!!! 걸린시간 : 3.23s
